# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [13]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Import API key
from api_keys import geoapify_key

In [14]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mulanje,-16.0316,35.5000,27.45,45,52,1.89,MW,1700845901
1,1,elne,42.6003,2.9715,17.17,64,0,8.75,FR,1700845901
2,2,adamstown,-25.0660,-130.1015,21.91,76,97,8.15,PN,1700845902
3,3,klyuchi,52.2667,79.1667,2.63,97,100,9.18,RU,1700845903
4,4,keflavik,64.0049,-22.5624,3.24,93,100,9.77,IS,1700845903


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [15]:
%%capture --no-display

# Configure the map plot
colors = city_data_df.City

# plt.figure(figsize = (20,10))

map1 = city_data_df.hvplot.points("Lng", "Lat", s = city_data_df.Humidity, c = colors, geo = True, tiles = "OSM", frame_height = 1000, frame_width = 2000)

# Save the map
# hvplot.save(map1, 'map1.html')
map1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (_color,_size)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [16]:
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mulanje,-16.0316,35.5000,27.45,45,52,1.89,MW,1700845901
1,1,elne,42.6003,2.9715,17.17,64,0,8.75,FR,1700845901
2,2,adamstown,-25.0660,-130.1015,21.91,76,97,8.15,PN,1700845902
3,3,klyuchi,52.2667,79.1667,2.63,97,100,9.18,RU,1700845903
4,4,keflavik,64.0049,-22.5624,3.24,93,100,9.77,IS,1700845903


In [17]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df2 = city_data_df.loc[((city_data_df["Max Temp"] <= 27)&(city_data_df["Max Temp"] > 21)) & (city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
city_data_df2 = city_data_df2.dropna(how = "any")

# Display sample data
city_data_df2.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
91,91,toliara,-23.3500,43.6667,25.87,86,0,3.08,MG,1700845990
243,243,dolores,-36.3132,-57.6792,26.92,16,0,2.76,AR,1700846126
260,260,jaumave,23.4167,-99.3833,24.07,42,0,1.71,MX,1700846143
270,270,carahue,-38.7000,-73.1667,23.20,44,0,3.02,CL,1700845856
504,504,tindouf,27.6711,-8.1474,21.25,19,0,1.54,DZ,1700846351
556,556,inhambane,-23.8650,35.3833,24.55,90,0,3.36,MZ,1700846394


### Step 3: Create a new DataFrame called `hotel_df`.

In [18]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df2.loc[:, ["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
91,toliara,MG,-23.3500,43.6667,86,
243,dolores,AR,-36.3132,-57.6792,16,
260,jaumave,MX,23.4167,-99.3833,42,
270,carahue,CL,-38.7000,-73.1667,44,
504,tindouf,DZ,27.6711,-8.1474,19,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [19]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
params = {
    'apiKey': geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row.Lat
    lng = row.Lng
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    params['categories'] = categories
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
toliara - nearest hotel: Ambary
dolores - nearest hotel: Plaza Hotel
jaumave - nearest hotel: No hotel found
carahue - nearest hotel: No hotel found
tindouf - nearest hotel: محمد بوسبي
inhambane - nearest hotel: Hotel de Inhambane


,City,Country,Lat,Lng,Humidity,Hotel Name
91,toliara,MG,-23.3500,43.6667,86,Ambary
243,dolores,AR,-36.3132,-57.6792,16,Plaza Hotel
260,jaumave,MX,23.4167,-99.3833,42,No hotel found
270,carahue,CL,-38.7000,-73.1667,44,No hotel found
504,tindouf,DZ,27.6711,-8.1474,19,محمد بوسبي
556,inhambane,MZ,-23.8650,35.3833,90,Hotel de Inhambane


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [20]:
# Configure the map plot
colors = hotel_df.City

map2 = hotel_df.hvplot.points("Lng", "Lat", s = hotel_df.Humidity, c = colors, geo = True, tiles = "OSM", hover_cols = ['Hotel Name', 'Country'])

# Display the map
map2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (_color,_size,Hotel Name,Country)